# 芒果品質分類模型

這個 Jupyter Notebook 會展示如何使用 PyTorch 和預訓練的 ResNet50 模型來訓練一個圖片分類器，以識別不同品質的芒果捏。

## 導入必要的庫

首先，我們需要導入 PyTorch 相關的庫以及 torchvision，後者提供了許多有用的工具和預訓練模型捏。


In [1]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader


## 設定圖片轉換

我們將設定轉換來預處理圖片資料捏。這包括調整圖片大小、轉換為張量以及標準化捏。


In [2]:
# 設定圖片轉換捏
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


## 加載訓練和驗證資料集

使用 ImageFolder 來加載訓練和驗證資料捏，這將自動為我們的圖片標籤化捏。


In [3]:
# 加載訓練和驗證資料捏
train_data = datasets.ImageFolder('dataset/train', transform=train_transform)
val_data = datasets.ImageFolder('dataset/valid', transform=val_transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)


## 建立和修改模型

我們將使用預訓練的 ResNet50 模型，並修改最後的全連接層以適應我們的三個類別捏。


In [4]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 3)  # 修改最後一層以匹配三個類別捏


c:\Users\Alan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Alan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## 設定裝置

確認是否可以使用 CUDA，並設定相應的裝置捏。


In [5]:
if torch.cuda.is_available():
    print("CUDA is available. Training on GPU.")
else:
    print("CUDA is not available. Training on CPU.")

# 設定裝置捏
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


CUDA is available. Training on GPU.


## 定義損失函數和優化器

為訓練過程設定損失函數和優化器捏。


In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


## 訓練模型

定義一個函數來執行訓練循環，包括前向傳播、損失計算、反向傳播和參數更新捏。


In [7]:
def train_model(model, criterion, optimizer, train_loader, val_loader, device, num_epochs=50):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # 調整學習率捏
        scheduler.step()

        # 驗證階段和輸出訓練資訊捏
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Epoch {epoch+1}, Train Loss: {running_loss / len(train_loader)}, '
            f'Val Loss: {val_loss / len(val_loader)}, '
            f'Accuracy: {100 * correct / total}%')



## 開始訓練和儲存模型

最後，我們執行訓練函數並儲存訓練好的模型捏。


In [8]:
# 開始訓練捏
train_model(model, criterion, optimizer, train_loader, val_loader, device)

# 儲存模型捏
torch.save(model.state_dict(), 'mango_classifier.pth')


Epoch 1, Train Loss: 0.8612371165411813, Val Loss: 0.6394688212871551, Accuracy: 70.5%
Epoch 2, Train Loss: 0.800097200870514, Val Loss: 0.7269153022766113, Accuracy: 64.875%
Epoch 3, Train Loss: 0.7866713152612959, Val Loss: 0.6318838155269623, Accuracy: 70.0%
Epoch 4, Train Loss: 0.7691187163761684, Val Loss: 0.651935625076294, Accuracy: 69.75%
Epoch 5, Train Loss: 0.7215755292347499, Val Loss: 0.6014129391312599, Accuracy: 71.375%
Epoch 6, Train Loss: 0.7413893069539751, Val Loss: 0.6797557258605957, Accuracy: 68.375%
Epoch 7, Train Loss: 0.7406436913354056, Val Loss: 0.6148427212238312, Accuracy: 71.375%
Epoch 8, Train Loss: 0.6943043972764696, Val Loss: 0.5765698802471161, Accuracy: 73.875%
Epoch 9, Train Loss: 0.6672165969439915, Val Loss: 0.5676766800880432, Accuracy: 75.0%
Epoch 10, Train Loss: 0.6763510348115649, Val Loss: 0.552786613702774, Accuracy: 76.125%
Epoch 11, Train Loss: 0.6711225712299347, Val Loss: 0.5421437650918961, Accuracy: 75.375%
Epoch 12, Train Loss: 0.65384